In [1]:
from __future__ import print_function
from __future__ import division
import json
import py_crepe
import datetime
import numpy as np
import data_helpers
import data
import string
import pandas as pd
np.random.seed(0123)  # for reproducibility


Using Theano backend.


In [2]:
# set parameters:

subset = None

#Whether to save model parameters
save = False
model_name_path = 'params/crepe_model.json'
model_weights_path = 'params/crepe_model_weights.h5'

#Maximum length. Longer gets chopped. Shorter gets padded.
maxlen = 1014

#Model params
#Filters for conv layers
nb_filter = 128 #initially 256
#Number of units in the dense layer
dense_outputs = 512 #Initially 1024
#Conv layer kernel size
filter_kernels = [7, 7, 3, 3, 3, 3]
#Number of units in the final output layer. Number of classes.

#Compile/fit params
batch_size = 32
nb_epoch = 70

In [3]:
print('Loading data...')
#Expect x to be a list of sentences. Y to be a one-hot encoding of the
#categories.

### 515-1122-122 and 1573 with remove 6 layers
#authorlist=[121, 479 , 649 ]
#doc_id = 14706

authorlist=[ 1078, 1087, 1416]
doc_id = [ 2501, 2502, 2700, 2701, 3259, 3257 ]
cat_output = len(authorlist) #binary in the last layer

# def main(authorlist, doc_id):
    
    
((trainX, trainY), (valX, valY)) = data_helpers.load_ag_data(authors = authorlist, docID = doc_id)

print('Creating vocab...')
vocab, reverse_vocab, vocab_size, check = data_helpers.create_vocab_set()


#trainX = data_helpers.encode_data(trainX, maxlen, vocab, vocab_size, check)
#test_data = data_helpers.encode_data(valX, maxlen, vocab, vocab_size, check)

print('Build model...')

classes = len(authorlist)
(model, sgd, model_weights_path) = py_crepe.build_model(classes, filter_kernels,
                                                        dense_outputs, maxlen, vocab_size, nb_filter)

Loading data...
Execution completed
Read completed
Number of rows: 105
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (52983, 2)
Author:  1078  Size:  4010
Author:  1087  Size: 15538
Author:  1416  Size: 33435
Min: 4010
Max: 33435
Authors [1078, 1087, 1416].
Found 12030 texts.
Found 12030 labels.
Creating vocab...
Build model...


In [4]:
vocab_size

500

In [5]:
print('Fit model...')
initial = datetime.datetime.now()
for e in xrange(nb_epoch):
    xi, yi = data_helpers.shuffle_matrix(trainX, trainY)
    xi_test, yi_test = data_helpers.shuffle_matrix(valX, valY)
    if subset:
        batches = data_helpers.mini_batch_generator(xi[:subset], yi[:subset],
                                                    vocab, vocab_size, check,
                                                    maxlen,
                                                    batch_size=batch_size)
    else:
        batches = data_helpers.mini_batch_generator(xi, yi, vocab, vocab_size,
                                                    check, maxlen,
                                                    batch_size=batch_size)

    test_batches = data_helpers.mini_batch_generator(xi_test, yi_test, vocab,
                                                     vocab_size, check, maxlen,
                                                     batch_size=batch_size)

    accuracy = 0.0
    loss = 0.0
    step = 1
    start = datetime.datetime.now()
    print('Epoch: {}'.format(e))
    for x_train, y_train in batches:
        
        f = model.train_on_batch(x_train, y_train)
        loss += f[0]
        loss_avg = loss / step
        accuracy += f[1]
        accuracy_avg = accuracy / step
        if step % 100 == 0:
            print('  Step: {}'.format(step))
            print('\tLoss: {}. Accuracy: {}'.format(loss_avg, accuracy_avg))
        step += 1

    test_accuracy = 0.0
    test_loss = 0.0
    test_step = 1
    
    for x_test_batch, y_test_batch in test_batches:
        f_ev = model.test_on_batch(x_test_batch, y_test_batch)
        test_loss += f_ev[0]
        test_loss_avg = test_loss / test_step
        test_accuracy += f_ev[1]
        test_accuracy_avg = test_accuracy / test_step
        test_step += 1
    stop = datetime.datetime.now()
    e_elap = stop - start
    t_elap = stop - initial
    print('Epoch {}. Loss: {}. Accuracy: {}\nEpoch time: {}. Total time: {}\n'.format(e, test_loss_avg, test_accuracy_avg, e_elap, t_elap))

if save:
    print('Saving model params...')
    json_string = model.to_json()
    with open(model_name_path, 'w') as f:
        json.dump(json_string, f)

model.save_weights(model_weights_path)

Fit model...
Epoch: 0
  Step: 100
	Loss: 1.03466424763. Accuracy: 0.45375
  Step: 200
	Loss: 0.90203497529. Accuracy: 0.54828125
  Step: 300
	Loss: 0.823189590971. Accuracy: 0.603333333333
Epoch 0. Loss: 0.547141330807. Accuracy: 0.776178727809
Epoch time: 0:06:12.654796. Total time: 0:06:12.913420

Epoch: 1
  Step: 100
	Loss: 0.522652076781. Accuracy: 0.7578125
  Step: 200
	Loss: 0.513188425899. Accuracy: 0.7609375
  Step: 300
	Loss: 0.502541306913. Accuracy: 0.767083333333
Epoch 1. Loss: 0.45341364059. Accuracy: 0.784676535349
Epoch time: 0:05:52.388172. Total time: 0:12:05.558098

Epoch: 2
  Step: 100
	Loss: 0.412180459946. Accuracy: 0.8240625
  Step: 200
	Loss: 0.402938581072. Accuracy: 0.82375
  Step: 300
	Loss: 0.385770478174. Accuracy: 0.832604166667
Epoch 2. Loss: 0.419714071445. Accuracy: 0.811266447368
Epoch time: 0:06:00.975220. Total time: 0:18:06.783919

Epoch: 3
  Step: 100
	Loss: 0.291129405946. Accuracy: 0.884375
  Step: 200
	Loss: 0.290412117075. Accuracy: 0.883125
  S

In [6]:
del trainX, trainY, valX, valY

In [7]:
model.load_weights(model_weights_path)

#from keras.optimizers import SGD
#sgd = SGD(lr=0.01, momentum=0.9, nesterov= True)

# Compile model again (required to make predictions)
model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])



In [10]:
test_binary = []
for docs in doc_id:
    (testX, testY) = data_helpers.load_doc_data(authors = authorlist, docID = docs)
    testX = data_helpers.encode_data(testX, maxlen, vocab, vocab_size, check)
    predY = np.array(model.predict(testX, batch_size=batch_size))
    testY = np.array(testY)
    predY = predY.mean(axis = 0)
    testY = testY.mean(axis = 0)
    predLocation = predY.tolist().index(max(predY))
    if predLocation == testY:
        test_binary.append(1)
    else:
        test_binary.append(0)
    
    from IPython.display import clear_output
    clear_output()
        



In [ ]:
#predY = np.array(model.predict(testX, batch_size=batch_size))

In [ ]:
#predY


In [ ]:
#predY.mean(axis = 0)

In [11]:
test_binary

[1, 1, 1, 1, 1, 1]

In [ ]:
"""
feature_model = py_crepe.build_feature_model()
feature_trainX = feature_model.predict(trainX)
feature_testX = feature_model.predict(testX)
"""